In [125]:
import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import recall_score
s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'churn-bigml-80.csv'
file_key2 = 'churn-bigml-20.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

bucket_object2 = bucket.Object(file_key2)
file_object2 = bucket_object2.get()
file_content_stream2 = file_object2.get('Body')

#reading the data-file
telecom_train = pd.read_csv(file_content_stream)
telecom_train.head()
telecom_test = pd.read_csv(file_content_stream2)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [126]:
#2b 
# replacing churn
telecom_train['Churn'].replace(['Yes', 'No'], [1,0], inplace=True)
telecom_test['Churn'].replace(['Yes', 'No'], [1,0], inplace=True)

#replacing international_plan
telecom_train['International_plan'].replace(['Yes', 'No'], [1,0], inplace=True)
telecom_test['International_plan'].replace(['Yes', 'No'], [1,0], inplace=True)

#replacing voice_mail_plan
telecom_train['Voice_mail_plan'].replace(['Yes', 'No'], [1,0], inplace=True)
telecom_test['Voice_mail_plan'].replace(['Yes', 'No'], [1,0], inplace=True)

telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [127]:
#2b cont 
#adding total_charge (sharpsightslab dataframe.assign(name = value))
telecom_train = telecom_train.assign(total_charge = sum([telecom_train['Total_day_charge'], telecom_train['Total_eve_charge'],
                                        telecom_train['Total_night_charge'], telecom_train['Total_intl_charge']]))
telecom_test = telecom_test.assign(total_charge = sum([telecom_test['Total_day_charge'], telecom_test['Total_eve_charge'],
                                        telecom_test['Total_night_charge'], telecom_test['Total_intl_charge']]))
telecom_train.head()


,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,total_charge
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False,75.56
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False,59.24
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False,62.29
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False,66.80
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False,52.09


In [128]:
# 2c 
# dropping the variables not wanted 
telecom_train = telecom_train.drop(columns = ['State', 'Area_code', 'Number_vmail_messages', 'Total_day_minutes', 'Total_day_calls', 
                                             'Total_day_charge', 'Total_eve_minutes', 'Total_eve_calls', 'Total_eve_charge', 'Total_night_minutes',
                                              'Total_night_calls', 'Total_night_charge', 'Total_intl_minutes', 'Total_intl_calls', 'Total_intl_charge'], axis = 1)
telecom_test = telecom_test.drop(columns = ['State', 'Area_code', 'Number_vmail_messages', 'Total_day_minutes', 'Total_day_calls', 
                                             'Total_day_charge', 'Total_eve_minutes', 'Total_eve_calls', 'Total_eve_charge', 'Total_night_minutes',
                                             'Total_night_calls', 'Total_night_charge', 'Total_intl_minutes', 'Total_intl_calls', 'Total_intl_charge'], axis = 1)
telecom_train.head()


,Account_length,International_plan,Voice_mail_plan,Customer_service_calls,Churn,total_charge
0,128,0,1,1,False,75.56
1,107,0,1,1,False,59.24
2,137,0,0,0,False,62.29
3,84,1,0,2,False,66.80
4,75,1,0,3,False,52.09


In [129]:
# 2d1
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'Customer_service_calls', 'total_charge']]
Y = telecom_train['Churn']


In [130]:
data = list()
for i in range (0,1000):
    # splitting data
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)
    
    # min-max transformation
    scaler = MinMaxScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    # estimating lanbda for lasso
    lasso_cv = LassoCV(normalize = True, cv= 5).fit(X_train, Y_train)
        
    # extracting the best lambda
    cv_lambda = lasso_cv.alpha_
    
    # building lasso model
    lasso_md = Lasso(alpha= cv_lambda, normalize = True).fit(X_train,Y_train)
    data.append(lasso_md.coef_)
    
data = pd.DataFrame(data)
data


,0,1,2,3,4
0,0.010752,0.313240,-0.089231,0.485134,0.511574
1,0.056089,0.331304,-0.074904,0.529298,0.494224
2,0.024322,0.299403,-0.066949,0.500118,0.508673
3,0.035561,0.308738,-0.080886,0.531908,0.549630
4,0.000000,0.312613,-0.067606,0.470798,0.522241
...,...,...,...,...,...
995,0.043844,0.303357,-0.075560,0.505530,0.534827
996,0.038134,0.312761,-0.070145,0.543310,0.485923
997,0.050178,0.320200,-0.079557,0.441342,0.488968
998,0.000000,0.333205,-0.071478,0.417569,0.446792


In [131]:
print(sum(data[0] == 0))
print(sum(data[1] == 0))
print(sum(data[2] == 0))
print(sum(data[3] == 0))
print(sum(data[4] == 0))
# drop the first one 

268
0
0
0
0


In [132]:
# removing ones more than 200 
telecom_train = telecom_train.drop(['Account_length'], axis = 1)
telecom_test = telecom_test.drop(['Account_length'], axis = 1)
telecom_test.head()

,International_plan,Voice_mail_plan,Customer_service_calls,Churn,total_charge
0,0,0,1,False,73.32
1,0,0,4,True,54.20
2,0,0,4,True,92.29
3,0,0,2,False,41.05
4,0,0,1,False,49.60


In [133]:
#2e
X2 = telecom_train[['International_plan', 'Voice_mail_plan', 'Customer_service_calls', 'total_charge']]
Y2 = telecom_train['Churn']

# lists for the results of each model 
md1res = list()
md2res = list()
md3res = list()
md4res = list()

for i in range (0,100):
    
    #split into folds 
    kfold= StratifiedKFold(n_splits=5, shuffle=True)
    
    #more lists for storing?
    md1foldres = list()
    md2foldres = list()
    md3foldres = list()
    md4foldres = list()
    
    # min-max transformation
    scaler = MinMaxScaler()
    X = pd.DataFrame(scaler.fit_transform(x))
    
    for trainix, valix in kfold.split(X2,Y2):
        #splitting data
        X2_train, X2_val = X2.iloc[trainix], X2.iloc[valix]
        Y2_train, Y2_val = Y2.iloc[trainix], Y2.iloc[valix]
    
        #######logistic model (1)
        logitmd1 = LogisticRegression(solver = 'linlinear', penalty = '11').fit(X_train2, Y_train)
        #predicting on test dataset
        logitmd1_pred = logitmd1.predict_proba(X2_test)[:,1]
        #cutoff (10%)
        logitmd1_label = np.where(logitmd1_pred < .10, 0,1)
        #recall score & store
        logitmd1_recall = recall_score(Y2_val, logitmd1_label)
        md1foldres.append(logitmd1_recall)
    

